Testing the cost function

In [1]:
from keras import backend as K
from keras.models import Model 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.optimizers import Adam, RMSprop

import time
import numpy as np
import h5py
import squeeze
import tensorflow as tf
import keras.backend as kb
import keras.losses as losses
import hf
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

Using TensorFlow backend.


In [2]:
json_dir = '/home/lmiguel/Projects/deep-learning/hyperface'

# data de entrenamiento normalizada
norm_train_data = hf.ImageDataGeneratorV2(featurewise_center=False,  # requires fit first
                                     featurewise_std_normalization=False,  # requires fit first
                                     samplewise_center=True,                                     
                                     samplewise_std_normalization=True)
norm_train_data_flow = norm_train_data.flow_from_directory(json_dir,
                                                 'positives.json', 'negatives.json', 
                                                 output_type='hyperface', target_size=(227, 227),
                                                 pos_batch_size=64, neg_batch_size=64)

# data de entrenamiento
train_data = hf.ImageDataGeneratorV2()
train_data_flow = train_data.flow_from_directory(json_dir,
                                                 'positives.json', 'negatives.json', 
                                                 # output_type='hyperface',
                                                 output_type='predict',
                                                 target_size=(227, 227),
                                                 pos_batch_size=64, neg_batch_size=64)

Found 2 positive samples and 2 negative samples.
Found 2 positive samples and 2 negative samples.


Pérdida para visibilidad

In [3]:
def custom_loss_visibility(y_true, y_pred):
    # en paper: loss = 1/N * sum((vpredi-vi)^2)
    # para entrenar:
    # return (1/21) * kb.sum(kb.square(y_pred-y_true), axis=-1)
    # para debug:
    return (1/21) * np.sum((y_pred-y_true)**2, axis=-1)

batch_x, [batch_y_fnf, batch_y_landmarks, batch_y_visfac, batch_y_pose, batch_y_gender] = norm_train_data_flow.next() 
vis = batch_y_visfac[0]
vis2 = np.zeros(vis.shape)
print(custom_loss_visibility(vis,vis2))
print(custom_loss_visibility(vis,vis))

0.809523809524
0.0


Pérdida para pose

In [4]:
def custom_loss_pose(y_true, y_pred):
    # en paper: loss = 1/3 * sum((ppredi-pi)^2)
    # para entrenamiento:
    # return (1/3) * kb.sum(kb.square(y_pred-y_true), axis=-1)
    # para debug:
    return (1/3) * np.sum((y_pred-y_true)**2, axis=-1)

batch_x, [batch_y_fnf, batch_y_landmarks, batch_y_visfac, batch_y_pose, batch_y_gender] = norm_train_data_flow.next() 
pose = batch_y_pose[0]
pose2 = np.zeros(pose.shape)
print(custom_loss_pose(pose,pose2))
print(custom_loss_pose(pose,pose))

0.191899001485
0.0


Pérdida para landmarks

In [6]:
def custom_loss_landmarks(coord_true, coord_pred):    
    # en paper: loss = 1/(2N) * Sum(vi*((xpredi-ai)^2 + (ypredi-bi)^2))
    x_true_coord = coord_true[0:21]
    y_true_coord = coord_true[21:42]
    viz_true = coord_true[42:63]
    x_pred_coord = coord_pred[0:21]
    y_pred_coord = coord_pred[21:42]
    # para entrenamiento:
    # return (1/(2*21)) * kb.sum(viz_true * (kb.square(x_pred_coord-x_true_coord) + K.square(y_pred_coord - y_true_coord)), axis=-1)
    # para debug:
    return (1/(2*21)) * np.sum(viz_true * ((x_pred_coord-x_true_coord)**2 + (y_pred_coord - y_true_coord)**2), axis=-1)

batch_x, [batch_y_fnf, batch_y_landmarks, batch_y_visfac, batch_y_pose, batch_y_gender] = norm_train_data_flow.next()
landmarks = batch_y_landmarks[0]
landmarks2 = np.zeros(landmarks.shape)
print(custom_loss_landmarks(landmarks, landmarks2))
print(custom_loss_landmarks(landmarks, landmarks))

0.030594314446
0.0


In [ ]:
json_dir = '/home/lmiguel/Projects/deep-learning/hyperface'
train_data = hf.ImageDataGeneratorV2(featurewise_center=False,  # requires fit first
                                     featurewise_std_normalization=False,  # requires fit first
                                     samplewise_center=True,                                     
                                     samplewise_std_normalization=True)

train_data_flow = train_data.flow_from_directory(json_dir,
                                                 'positives.json', 'negatives.json', 
                                                 output_type='hyperface', target_size=(227, 227),
                                                 pos_batch_size=64, neg_batch_size=64)

batch_x, [batch_y_fnf, batch_y_landmarks, batch_y_visfac, batch_y_pose, batch_y_gender] = train_data_flow.next()
# print( batch_y_landmarks)
landmarks_true = batch_y_landmarks[0]
# print(landmarks_true)
print(batch_x[0])
landmarks_pred = np.zeros(landmarks_true.shape)
cost = custom_loss_landmarks(landmarks_true, landmarks_pred)
print(cost)
cost = custom_loss_landmarks(landmarks_true, landmarks_true)
print(cost)


# img = batch_x[0]
# landmarks = np.dstack([landmarks_true[0:21],landmarks_true[21:42]])[0]
# for pt in landmarks:
#     pt = (int(pt[0]), int(pt[1]))
#     cv2.circle(img, pt, 4, (255,255,255), -1)
# plt.figure()
# plt.imshow(img)


train_data = hf.ImageDataGeneratorV2()
train_data_flow = train_data.flow_from_directory(json_dir,
                                                 'positives.json', 'negatives.json', 
                                                 # output_type='hyperface',
                                                 output_type='predict',
                                                 target_size=(227, 227),
                                                 pos_batch_size=64, neg_batch_size=64)
# batch_x, [batch_y_fnf, batch_y_landmarks, batch_y_visfac, batch_y_pose, batch_y_gender] = train_data_flow.next()
batch_x, _, batch_bbox, batch_y_fnf, batch_y_landmarks, batch_y_visfac, batch_y_pose, batch_y_gender = train_data_flow.next()
landmarks_true = batch_y_landmarks[0]
# landmarks_pred = np.zeros(landmarks_true.shape)
# print(landmarks_true)
# print(batch_bbox[0])

# path = '/home/lmiguel/Projects/datasets/aflw/aflw/data/flickr/3/image00035.jpg'
# img = cv2.imread(path)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# landmarks = np.dstack([landmarks_true[0:21],landmarks_true[21:42]])[0]
# for pt in landmarks:
#     pt = (int(pt[0]), int(pt[1]))
#     cv2.circle(img, pt, 4, (255,255,255), -1)
# plt.figure()
# plt.imshow(img)



In [3]:
a = list([1])
n = np.array([2,3])
a.extend(n)
# print(a)
np.array(a)
np.append(n,a)

array([2, 3, 1, 2, 3])

In [ ]:
# other custom loss wich are wrong!
# loss from hyperface-with-squeezenet
def custom_mse_lm(y_true,y_pred):
    return kb.sign(kb.sum(kb.abs(y_true),axis=-1))*kb.sum(kb.square(tf.multiply((kb.sign(y_true)+1)*0.5, y_true-y_pred)),axis=-1)/kb.sum((kb.sign(y_true)+1)*0.5,axis=-1)

def custom_mse_pose(y_true,y_pred):
    return kb.sign(kb.sum(kb.abs(y_true),axis=-1))*losses.mean_squared_error(y_true,y_pred)

def custom_mse_lm(y_true,y_pred):
    return kb.sign(kb.sum(kb.abs(y_true),axis=-1))* kb.sum(kb.square(tf.multiply((kb.sign(y_true)+1)*0.5, y_true-y_pred)),axis=-1)/kb.sum((kb.sign(y_true)+1)*0.5,axis=-1)

def mean_squared_error(y_true, y_pred):
    return kb.mean(kb.square(y_pred - y_true), axis=-1)